In [3]:
# !pip install langchain-pinecone

  Using cached pinecone-7.3.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached pinecone_plugin_assistant-1.8.0-py3-none-any.whl.metadata (30 kB)
Using cached pinecone-7.3.0-py3-none-any.whl (587 kB)
Using cached pinecone_plugin_assistant-1.8.0-py3-none-any.whl (259 kB)

  Attempting uninstall: pinecone-plugin-assistant

    Found existing installation: pinecone-plugin-assistant 3.0.1

    Uninstalling pinecone-plugin-assistant-3.0.1:

      Successfully uninstalled pinecone-plugin-assistant-3.0.1

   -------- ------------------------------- 1/5 [pinecone-plugin-assistant]
  Attempting uninstall: pinecone
   -------- ------------------------------- 1/5 [pinecone-plugin-assistant]
    Found existing installation: pinecone 8.0.0
   -------- ------------------------------- 1/5 [pinecone-plugin-assistant]
   ---------------- ----------------------- 2/5 [pinecone]
    Uninstalling pinecone-8.0.0:
   ---------------- ----------------------- 2/5 [pinecone]
      Successfully uninstalled pinec

In [4]:
import os
import pandas as pd

# LangChain
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_core.documents import Document

# Pinecone
from pinecone import Pinecone, ServerlessSpec

# env
from dotenv import load_dotenv
load_dotenv()

import json
import time
from pathlib import Path
from openai import OpenAI

In [20]:
INDEX_NAME = "funeral-services"
NAMESPACE = "legacy"
PROCESSED_DIR = "../data/processed"

In [30]:
pc = Pinecone()
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
openai_client = OpenAI()
print(index.describe_index_stats(), '\n\n')

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'digital_legacy': {'vector_count': 68},
                'funeral_facilities': {'vector_count': 2518},
                'guide': {'vector_count': 4},
                'ordinance': {'vector_count': 234}},
 'total_vector_count': 2824,
 'vector_type': 'dense'} 




In [31]:
def setup_index():
    """Pinecone 인덱스 확인"""
    existing_indexes = [index.name for index in pc.list_indexes()]
    
    if INDEX_NAME not in existing_indexes:
        print(f"오류: 인덱스 '{INDEX_NAME}'가 존재하지 않습니다.")
        return None
    
    print(f"기존 인덱스 '{INDEX_NAME}' 사용 (Namespace: {NAMESPACE})")
    return pc.Index(INDEX_NAME)

In [32]:
def get_embedding(text: str, model: str = "text-embedding-3-small") -> list:
    """텍스트를 임베딩 벡터로 변환"""
    try:
        response = openai_client.embeddings.create(
            model=model,
            input=text
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"임베딩 생성 오류: {e}")
        raise

In [33]:
import json
import time
import uuid # 고유 ID 생성을 위해 추가

# (get_embedding 함수 등 상단 부분은 동일)

def upsert_pinecone(file_path):
    vectors = [] 
    count = 0   

    print(f"📂 파일 처리 시작: {file_path}")

    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        total_lines = len(lines)
        
        for i, line in enumerate(lines):
            if not line.strip():
                continue
                
            try:
                data = json.loads(line)
            except json.JSONDecodeError:
                print(f"⚠️ JSON 파싱 에러 (Line {i}) -> 건너뜀")
                continue

            # [수정 1] ID 처리 강화: ID가 없으면 임시 ID 생성
            doc_id = data.get('id')
            if not doc_id:
                doc_id = f"doc_{i}_{str(uuid.uuid4())[:8]}" 
            
            title = data.get('title', '')
            raw_text = data.get('text', '') # 원본 텍스트는 따로 저장
            source = data.get('source', '')
            category = data.get('category', '')
            article_id = data.get('article_id', '')
            article_title = data.get('article_title', '')

            # [수정 2] 텍스트 구성 중복 제거 및 공백 추가
            # 기존 코드의 text = title + text 삭제함
            full_text = f"{title} {raw_text}".strip()

            # 텍스트가 비어있으면 임베딩 불가하므로 스킵
            if not full_text:
                continue

            # 메타데이터 구성
            metadata = {
                'title': title,
                'source': source,
                'category': category,
                'article_id' : article_id,
                'article_title' :article_title,
                'text': raw_text # 원본 텍스트도 메타데이터에 포함 (선택 사항)
            }
        
            # 임베딩 생성
            try:
                embedding = get_embedding(full_text)
                vectors.append({
                    "id": doc_id,          
                    "values": embedding,   
                    "metadata": metadata   
                })
            except Exception as e:
                print(f"❌ 임베딩 생성 오류 (ID: {doc_id}): {e}")
                continue
            
            # 배치 업로드
            if len(vectors) >= 100:
                try:
                    index.upsert(vectors=vectors, namespace=NAMESPACE)
                    count += len(vectors)
                    print(f"🚀 진행률: {count}/{total_lines} ({(count/total_lines)*100:.1f}%)")
                    vectors = [] 
                    time.sleep(0.5) 
                except Exception as e:
                    print(f"🔥 배치 업로드 오류: {e}")
        
        # 남은 벡터 업로드
        if vectors:
            try:
                index.upsert(vectors=vectors, namespace=NAMESPACE)
                count += len(vectors)
                print(f"✅ 최종 완료: {count}/{total_lines}")
            except Exception as e:
                print(f"🔥 마지막 배치 업로드 오류: {e}")

# 실행 부분
data_files = [
    "../data/processed/1_minbeob_sangsok_chunks.jsonl",
    "../data/processed/2_segeumsangsik_I_simple.jsonl",
    "../data/processed/3_segeumsangsik_II_simple.jsonl",
    "../data/processed/4_ansimsangsok_web_simple.jsonl",
    "../data/processed/5_jaesanjohoe_rule_chunks.jsonl",
    "../data/processed/6_sangsokse_beob_chunks.jsonl"
]

for data_file in data_files:
    try:
        upsert_pinecone(data_file)
    except Exception as e:
        print(f"파일 처리 중 오류 발생 ({data_file}): {e}")

📂 파일 처리 시작: ../data/processed/1_minbeob_sangsok_chunks.jsonl
🚀 진행률: 100/129 (77.5%)
✅ 최종 완료: 129/129
📂 파일 처리 시작: ../data/processed/2_segeumsangsik_I_simple.jsonl
✅ 최종 완료: 22/22
📂 파일 처리 시작: ../data/processed/3_segeumsangsik_II_simple.jsonl
✅ 최종 완료: 23/23
📂 파일 처리 시작: ../data/processed/4_ansimsangsok_web_simple.jsonl
✅ 최종 완료: 3/3
📂 파일 처리 시작: ../data/processed/5_jaesanjohoe_rule_chunks.jsonl
✅ 최종 완료: 25/25
📂 파일 처리 시작: ../data/processed/6_sangsokse_beob_chunks.jsonl
🚀 진행률: 100/248 (40.3%)
🚀 진행률: 200/248 (80.6%)
✅ 최종 완료: 248/248
